In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary


# Step:- 2 | Modelling the componets(dataset and Architecture)
## 1st Dataset component 

In [11]:
import os
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms

# this dataset class should take a path, which is we have fixed using set_default_config of parameter train_data_src and valid_data_src
# in our case real and fake images are in different folders, so the Dataset class should be made in such a way.
# also 
class DS01(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.transform = transforms.Compose([
                            transforms.Resize((128,128)),  # Resize images 
                            transforms.ToTensor()        # Convert images to tensor
                        ])
        self.image_paths = []
        self.labels = []
        
        # Map directories with 'real' or 'fake' in their name to corresponding labels
        self.class_to_idx = {'real': 1, 'fake': 0}  # 1 for real, 0 for fake
        
        # Traverse through subdirectories
        for subdir in os.listdir(root_dir):
            class_name = None
            
            # Identify if subdir is 'real' or 'fake' by checking the directory name
            if 'real' in subdir.lower():
                class_name = 'real'
            elif 'fake' in subdir.lower():
                class_name = 'fake'
            
            if class_name:
                class_dir = os.path.join(root_dir, subdir)
                if os.path.isdir(class_dir):
                    for img_name in os.listdir(class_dir):
                        img_path = os.path.join(class_dir, img_name)
                        # Only load .jpg files
                        if img_name.lower().endswith('.jpg'):
                            self.image_paths.append(img_path)
                            self.labels.append(self.class_to_idx[class_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        
        # Open image
        image = Image.open(img_path).convert("RGB")
        
        # Apply transforms if provided
        image = self.transform(image)
        
        return image, label



### checking input size for the model that is returned from DS01

In [12]:
# creating an instance of DS01 for testing
dataset = DS01(root_dir="../DataSet/real_vs_fake/real_vs_fake/train/")
# DataLoader for batching
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
dl = iter(dataloader)
X,y = next(dl)
# X and y are the input and target of first batch 
# so we take 1 input and its target to check their sizes
X[0].shape,y[0]

(torch.Size([3, 128, 128]), tensor(0))

***Here [3, 128, 128] is the input size*** so model should take input size of the same

In [13]:
# A CNN Model
class testCNN(nn.Module):
    def __init__(self):
        super(testCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(8)
        self.pool = nn.MaxPool2d(kernel_size=2,stride=2,padding=0)
        self.fc1 = nn.Linear(8*64*64, 16)
        self.fc2 = nn.Linear(16, 1)
        self.dropout = nn.Dropout(0.5)
    def forward(self, x):
        x = x.to(next(self.parameters()).device) # to assign the input to the same device,
#         torchsummary.summary sets model to gpu but for input it does not
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = x.view(x.size(0), -1)  # Flatten feature maps
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.sigmoid(self.fc2(x))
        return x
model1 = testCNN()
summary(model1,input_size=(3,128,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 8, 128, 128]             224
       BatchNorm2d-2          [-1, 8, 128, 128]              16
         MaxPool2d-3            [-1, 8, 64, 64]               0
            Linear-4                   [-1, 16]         524,304
           Dropout-5                   [-1, 16]               0
            Linear-6                    [-1, 1]              17
Total params: 524,561
Trainable params: 524,561
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.19
Forward/backward pass size (MB): 2.25
Params size (MB): 2.00
Estimated Total Size (MB): 4.44
----------------------------------------------------------------


***From the avobe Total params is 524,561.*** 
### Now we have to  check the compactibility of model and dataset components
Call `test_mods` if your default_config is set ( using `set_default_config` that i have done in `training.ipynb` file) then you only have to set the parametrs `model` and `dataset`.

In [15]:
from PyTorchLabFlow import test_mods
P = test_mods(dataset=DS01,model=model1,prepare=True)
P.train(num_epochs=2)

Configuration file is saved at internal/Test/test_c.json
History will be saved at internal/Test/test_h.csv
Weights will be saved at internal/Test/test_w.pth
Data loaders are successfully created


Epoch 1/2: 100%|█| 3125/3125 [1:04:24<00:00,  1.24s/it, accuracy=0.5, loss=0.69


Epoch 1, Train Loss: 0.6957, Train Accuracy: 0.50, Val Loss: 0.6931, Val Accuracy: 0.50
Best Model Weights Updated: Epoch 1 - Val Loss: 0.6931499324256568


Epoch 2/2: 100%|█| 3125/3125 [04:18<00:00, 12.07it/s, accuracy=0.501, loss=0.69
                                                                               

Epoch 2, Train Loss: 0.6932, Train Accuracy: 0.50, Val Loss: 0.6932, Val Accuracy: 0.50
Finished Training


If training successfully started then model and dataset classes are compactible to eachother and if not then you should check according to error.

If everything goes wel we should move for the next steps

### Place the components at the respective files
copy the whole code of model class, in our case it is
```python
class testCNN(nn.Module):
    def __init__(self):
        super(testCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(8)
        self.pool = nn.MaxPool2d(kernel_size=2,stride=2,padding=0)
        self.fc1 = nn.Linear(8*64*64, 16)
        self.fc2 = nn.Linear(16, 1)
        self.dropout = nn.Dropout(0.5)
    def forward(self, x):
        x = x.to(next(self.parameters()).device) # to assign the input to the same device,
#         torchsummary.summary sets model to gpu but for input it does not
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = x.view(x.size(0), -1)  # Flatten feature maps
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.sigmoid(self.fc2(x))
        return x
```
copy thye avobe and paste inside `Libs/models.py`
and copy the dataset class 
```python
class DS01(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.transform = transforms.Compose([
                            transforms.Resize((128,128)),  # Resize images 
                            transforms.ToTensor()        # Convert images to tensor
                        ])
        self.image_paths = []
        self.labels = []
        
        # Map directories with 'real' or 'fake' in their name to corresponding labels
        self.class_to_idx = {'real': 1, 'fake': 0}  # 1 for real, 0 for fake
        
        # Traverse through subdirectories
        for subdir in os.listdir(root_dir):
            class_name = None
            
            # Identify if subdir is 'real' or 'fake' by checking the directory name
            if 'real' in subdir.lower():
                class_name = 'real'
            elif 'fake' in subdir.lower():
                class_name = 'fake'
            
            if class_name:
                class_dir = os.path.join(root_dir, subdir)
                if os.path.isdir(class_dir):
                    for img_name in os.listdir(class_dir):
                        img_path = os.path.join(class_dir, img_name)
                        # Only load .jpg files
                        if img_name.lower().endswith('.jpg'):
                            self.image_paths.append(img_path)
                            self.labels.append(self.class_to_idx[class_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        
        # Open image
        image = Image.open(img_path).convert("RGB")
        
        # Apply transforms if provided
        image = self.transform(image)
        
        return image, label
```
and paste it inside `Libs/datasets.py`

## Now time to go to `training.ipynb`
# check there for further proccesses